In [1]:
import pandas as pd
import re

## First ETL

### Function to clean strings:

In [2]:
def string_cleaner(dataframe,col):
    
    """
    Remove all special characters, remove '&' and '-', remove double spaces and end and start string spaces.
    Return the cleaned Dataframe
    
    Inputs:
    
    dataframe : A pandas dataframe
    col : A collumn name as string
    
    """
    df[col] = df[col].str.lower().replace('[^a-zA-Z0-9]', '')
    df[col] = df[col].str.replace('[&-/]', ' ')
    df[col] = df[col].str.replace('  ', ' ')
    df[col] = df[col].str.strip()
    return df

In [3]:
#Original DF
df = pd.read_csv('../data/merged_database/combined.csv' , index_col=0, dtype={'Quantity': str, 'Price': str})

In [4]:
#Original size:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137389 entries, 0 to 137388
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Manufacturer  137256 non-null  object
 1   Partnumber    137320 non-null  object
 2   Quantity      136526 non-null  object
 3   Price         137377 non-null  object
 4   supplier      137389 non-null  object
dtypes: object(5)
memory usage: 6.3+ MB


In [5]:
# Cleaning the two string cols:
string_cleaner(df,'Manufacturer')
string_cleaner(df,'Partnumber')

C:\Users\eppmi\AppData\Local\Temp/ipykernel_7548/2514458280.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace('[&-/]', ' ')


,Manufacturer,Partnumber,Quantity,Price,supplier
0,frisart,40004,1,27.34,metal
1,sanfil,5045,1,52.58,metal
2,branil,bcht004a,1,12.3,metal
3,branil,bcho108,1,22.09,metal
4,branil,bchv101rv,1,53.15,metal
...,...,...,...,...,...
137384,3 rho,7735,3,155.2132,sueyasu
137385,3 rho,7737,5,146.234,sueyasu
137386,3 rho,7744,2,153.9712,sueyasu
137387,3 rho,7748,1,188.93120000000002,sueyasu


### Transforming the price col:

In [6]:
df['Price'] = df['Price'].astype(str)
df['Price'] = df['Price'].apply(lambda x: x.replace(',','.'))
df['Price'] = pd.to_numeric(df['Price'],errors='coerce').round(2)

### Dics to alter the quantity col:

In [7]:
compel = {"Indisponível" : '0',
'A': '3',
'B' : '15',
 'C': '20'}

In [8]:
real = {'A' : '4',
'B' : '11'}

In [9]:
# Changing compel
df[df.supplier == 'compel'] = df[df.supplier == 'compel'].replace(compel, regex=True)

In [10]:
# Changing real
df[df.supplier == 'real'] = df[df.supplier == 'real'].replace(compel, regex=True)

### Modifying the Quantity col:

In [11]:
# Replace '>' and white spaces
df.Quantity = df.Quantity.str.replace('>', '')
df.Quantity = df.Quantity.str.strip()

In [12]:
#transforming to numeric
df.Quantity =pd.to_numeric(df['Quantity'],errors='coerce')

In [13]:
# Creating the key col:
df['key'] = df.Manufacturer+df.Partnumber

In [14]:
#Creating the pointer col:
df['pointer'] = df.supplier+df.Manufacturer+df.Partnumber

In [15]:
# Size of the first etl dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137389 entries, 0 to 137388
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Manufacturer  137256 non-null  object 
 1   Partnumber    137320 non-null  object 
 2   Quantity      136526 non-null  float64
 3   Price         137192 non-null  float64
 4   supplier      137389 non-null  object 
 5   key           137217 non-null  object 
 6   pointer       137217 non-null  object 
dtypes: float64(2), object(5)
memory usage: 8.4+ MB


In [16]:
# Partnumber as string
df.Partnumber = df.Partnumber.astype(str)

In [17]:
# Removing the dots in Partnumber
df.Partnumber = df.Partnumber.str.replace(".","")

C:\Users\eppmi\AppData\Local\Temp/ipykernel_7548/1128347612.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.Partnumber = df.Partnumber.str.replace(".","")


In [18]:
#Removing white space in Partnumber
df.Partnumber = df.Partnumber.str.strip()
df.Partnumber = df.Partnumber.str.replace(" ","")

In [19]:
# Manufacturer as string
df.Manufacturer = df.Manufacturer.astype(str)

In [20]:
# Removing the dots in Partnumber
df.Manufacturer = df.Manufacturer.str.replace(".","")

C:\Users\eppmi\AppData\Local\Temp/ipykernel_7548/3962159109.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.Manufacturer = df.Manufacturer.str.replace(".","")


In [21]:
#Removing white space in Partnumber
df.Manufacturer = df.Manufacturer.str.strip()
df.Manufacturer = df.Manufacturer.str.replace(" ","")

In [22]:
# Fill Nans with 0
df = df.replace("", 0)
df = df.replace("nan",0)
df = df.fillna(0)

In [23]:
# Removing empty Manufactures
df = df[df.Manufacturer != 0]

In [24]:
df.to_csv('../data/etl/first_etl.csv')

In [25]:
#Save with timestamp
df.to_csv('../data/text_output/first_etl{}.txt'.format(pd.datetime.now().strftime("%Y-%m-%d %H-%M-%S")))

C:\Users\eppmi\AppData\Local\Temp/ipykernel_7548/4118386849.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df.to_csv('../data/text_output/first_etl{}.txt'.format(pd.datetime.now().strftime("%Y-%m-%d %H-%M-%S")))


## Second ETL (for each step i show the size of the modifield dataframe)

In [26]:
#parts having a quantity equal to 0

df = df[df['Quantity'] !=0]

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118212 entries, 0 to 137388
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Manufacturer  118212 non-null  object 
 1   Partnumber    118212 non-null  object 
 2   Quantity      118212 non-null  float64
 3   Price         118212 non-null  float64
 4   supplier      118212 non-null  object 
 5   key           118212 non-null  object 
 6   pointer       118212 non-null  object 
dtypes: float64(2), object(5)
memory usage: 7.2+ MB


In [28]:
df.supplier.value_counts()

metal      31276
rufato     18201
carbwel    15856
real       15113
lucios     14054
medauto     7494
compel      6882
sueyasu     5197
ima         1641
mte         1309
jahu        1189
Name: supplier, dtype: int64

In [29]:
#parts having the quantity blank

df = df[df['Quantity'].isna() == False]

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118212 entries, 0 to 137388
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Manufacturer  118212 non-null  object 
 1   Partnumber    118212 non-null  object 
 2   Quantity      118212 non-null  float64
 3   Price         118212 non-null  float64
 4   supplier      118212 non-null  object 
 5   key           118212 non-null  object 
 6   pointer       118212 non-null  object 
dtypes: float64(2), object(5)
memory usage: 7.2+ MB


In [31]:
df.supplier.value_counts()

metal      31276
rufato     18201
carbwel    15856
real       15113
lucios     14054
medauto     7494
compel      6882
sueyasu     5197
ima         1641
mte         1309
jahu        1189
Name: supplier, dtype: int64

In [32]:
#parts having a negative quantity

df = df[df['Quantity'] > 0]

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118212 entries, 0 to 137388
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Manufacturer  118212 non-null  object 
 1   Partnumber    118212 non-null  object 
 2   Quantity      118212 non-null  float64
 3   Price         118212 non-null  float64
 4   supplier      118212 non-null  object 
 5   key           118212 non-null  object 
 6   pointer       118212 non-null  object 
dtypes: float64(2), object(5)
memory usage: 7.2+ MB


In [34]:
#parts having a price equal to 0

df = df[df['Price'] != 0.]

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117382 entries, 0 to 137388
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Manufacturer  117382 non-null  object 
 1   Partnumber    117382 non-null  object 
 2   Quantity      117382 non-null  float64
 3   Price         117382 non-null  float64
 4   supplier      117382 non-null  object 
 5   key           117382 non-null  object 
 6   pointer       117382 non-null  object 
dtypes: float64(2), object(5)
memory usage: 7.2+ MB


In [36]:
#parts having the price blank
df = df[df['Price'].isna() == False]

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117382 entries, 0 to 137388
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Manufacturer  117382 non-null  object 
 1   Partnumber    117382 non-null  object 
 2   Quantity      117382 non-null  float64
 3   Price         117382 non-null  float64
 4   supplier      117382 non-null  object 
 5   key           117382 non-null  object 
 6   pointer       117382 non-null  object 
dtypes: float64(2), object(5)
memory usage: 7.2+ MB


In [38]:
#parts having the manufacturer blank
df = df[df['Manufacturer'].isna() == False]

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117382 entries, 0 to 137388
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Manufacturer  117382 non-null  object 
 1   Partnumber    117382 non-null  object 
 2   Quantity      117382 non-null  float64
 3   Price         117382 non-null  float64
 4   supplier      117382 non-null  object 
 5   key           117382 non-null  object 
 6   pointer       117382 non-null  object 
dtypes: float64(2), object(5)
memory usage: 7.2+ MB


In [40]:
# parts have the partnumber blank
df = df[df['Partnumber'].isna() == False]

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117382 entries, 0 to 137388
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Manufacturer  117382 non-null  object 
 1   Partnumber    117382 non-null  object 
 2   Quantity      117382 non-null  float64
 3   Price         117382 non-null  float64
 4   supplier      117382 non-null  object 
 5   key           117382 non-null  object 
 6   pointer       117382 non-null  object 
dtypes: float64(2), object(5)
memory usage: 7.2+ MB


In [42]:
# Read the synonyms data
syn = pd.read_excel("../data/synonyms/manufacturers_synonyms.xlsx")

In [43]:
# for each entrie replace the values with the syn dataframe:
for n,i in enumerate(syn.name_to_be_replaced):
    df = df.replace(i,syn.iloc[n,1])

In [44]:
df.to_csv('../data/etl/second_etl.csv')

In [45]:
#Save with timestamp
df.to_csv('../data/text_output/second_etl{}.txt'.format(pd.datetime.now().strftime("%Y-%m-%d %H-%M-%S")))

C:\Users\eppmi\AppData\Local\Temp/ipykernel_7548/93840401.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df.to_csv('../data/text_output/second_etl{}.txt'.format(pd.datetime.now().strftime("%Y-%m-%d %H-%M-%S")))


In [46]:
# save in excel
df.to_excel('../data/etl/second_etl.xlsx')